In [230]:
import numpy as np
import copy
import itertools

In [231]:
class Proportionality:
    def __init__(self, type_p, id1, id2):
        '''
        type_p = [-, +]
        id1 has P[type_p] on id2
        '''
        self.type = type_p
        self.id1 = id1
        self.id2 = id2
        
class Influence:
    def __init__(self, type_i, id1, id2):
        '''
        type_i = [-, +]
        id1 has I[type_p] on id2
        '''
        self.type = type_i
        self.id1 = id1
        self.id2 = id2
        
        

In [232]:
P1 = Proportionality('+', 2, 3)
#P2 = Proportionality('+', 2, 4)
#P3 = Proportionality('+', 4, 5)

I1 = Influence('-', 3, 2)
I2 = Influence('+', 1, 2)

list_influence = [I1, I2]
list_proportionality = [P1]

In [233]:
class variable:
    def __init__(self, id_state, list_proportionality, list_influence, possible_magnitude, possible_derivative, impossible_states, name):
        '''
        impossible_states - list of lists with impossible combinatiions of magnitude and derivative
        '''
        self.id = id_state
        self.proportionality = list_proportionality
        self.influence = list_influence
        self.possible_magnitude = possible_magnitude
        self.possible_derivative = possible_derivative
        self.impossible_states = impossible_states
        self.name = name
        self.magnitude = 'not assigned'
        self.derivative = 'not assigned'
    
    def print_state(self):
        print(self.name, ' M: ', self.magnitude,' D: ', self.derivative)
        
    
    def check_magnitude(self):
        if (self.magnitude == 2 and self.derivative == 1) or (self.magnitude == 0 and self.derivative == -1):
            return False
        else: 
            return True
            
            
            
    def assign_value(self, magnitude, derivative):
        if magnitude in self.possible_magnitude and derivative in self.possible_derivative and [magnitude, derivative] not in self.impossible_states:
            self.magnitude = magnitude
            self.derivative = derivative
            return True
        else:
            #print('Error')
            return False
            
            
class system:
    def __init__(self, v):
        '''
        s - list of states
        '''
        #self.id = id1
        self.vc = []
        self.variables = {}
        
        self.connection_to = set()
        self.connection_from = set()
        for k in v.keys():
            self.variables[k] = copy.deepcopy(v[k])

    
    def add_variable(self, v):
        self.variables[v.id] = copy.deepcopy(v)
        
    def add_id(self, id1):
        self.id = id1

    
    def print_system(self):
        print('SYSTEM: ')
        for s in self.variables:
            self.variables[s].print_state()
            
    def full_print(self):
        print('SYSTEM: ')
        for s in self.variables:
            self.variables[s].print_state()
            
        print('From ', self.id, ' such transitions are possible: ', self.connection_to)
        
        print('To ', self.id, ' are possible transitions from : ', self.connection_from)
        
            
    def add_vc(self, id1, id2, type_vc, value):
        '''
        dictionary with id1, id2, and name of the value which should be equal and the exact value 
        {'id1':id1, 'id2':id2, type_vc = 'magnitude', value = 2}
        '''
        vc_constr = {'id1':id1, 'id2':id2, 'type_vc' :type_vc, 'value': value}
        
        self.vc.append(vc_constr)
        
    def add_connection_to(self, c_t):
        self.connection_to.add(c_t)
    
    def add_connection_from(self, f_t):
        self.connection_from.add(f_t)
        
    def check_vc(self):
        if self.vc == []:
            return True
        
        for constr in self.vc:
            id1 = constr['id1']
            id2 = constr['id2']
            value = constr['value']
            if constr['type_vc'] == 'magnitude':
                if (self.variables[id1].magnitude == value) != (self.variables[id2].magnitude == value):
                    return False
                
            if constr['type_vc'] == 'derivative':
                if (self.variables[id1].derivative == value) != (self.variables[id2].derivative == value):
                    return False
        
        return True

                
    def check_system(self):
        
        for var in self.variables.values():
            if not var.check_magnitude():
                return False
            if [var.magnitude, var.derivative] in var.impossible_states:
                return False

            
        if not(self.check_vc()):
            return False
        
        possible = 0
        
        for var in self.variables.values():
            
            
            ID = var.id
            DER = var.derivative
            
            p_plus = [p.id1 for p in list_proportionality if p.type == '+' and p.id2 == ID]
            p_minus = [p.id1 for p in list_proportionality if p.type == '-' and p.id2 == ID]

            l_plus = [l.id1 for l in list_influence if l.type == '+' and l.id2 == ID]
            l_minus = [l.id1 for l in list_influence if l.type == '-' and l.id2 == ID]
            
            if len(p_plus) == 0 and len(p_minus) == 0 and len(l_minus) == 0 and len(l_plus) == 0:
                possible += 1
                continue
            
            
            possible_states = set()
            
            for id1 in p_plus:
                d = self.variables[id1].derivative
                if d == 1:
                    possible_states.add(1)
                    possible_states.add(0)
                    #possible_states.add(-1)
                #elif d == -1:
                #    possible_states.add(-1)
                #    possible_states.add(0)
                

            for id1 in p_minus:
                d = self.variables[id1].derivative
                if d == 1:
                    possible_states.add(-1)
                    possible_states.add(0)
                    #possible_states.add(1)
                #elif d == -1:
                #    possible_states.add(1)
                #    possible_states.add(0)
                    
            for id1 in l_plus:
                m = self.variables[id1].magnitude
                if m == 1:
                    possible_states.add(1)
                    possible_states.add(0)
                #if m == -1:
                #    possible_states.add(-1)

            for id1 in l_minus:
                m = self.variables[id1].magnitude
                if m == 1:
                    possible_states.add(-1)
                    possible_states.add(0)
                #if m == -1:
                #    possible_states.add(1)
                    
            if len(possible_states) == 0:
                possible_states.add(0)

            if 1 in possible_states and -1 in possible_states:
                possible_states.add(0)
   
            if DER not in possible_states:
                return False
        
            if DER in possible_states:
                possible += 1
        
        if possible == len(self.variables):
            return True
        
        return False
    
    


In [234]:
def check_continuity(s1, s2):
        '''
        from s1 to s2 
        '''
        for var in s1.variables.keys():
            if s1.variables[var].magnitude > 1 and s2.variables[var].magnitude < 1:
                return False
            
            if s1.variables[var].magnitude < 1 and s2.variables[var].magnitude > 1:
                return False
            
            if s1.variables[var].derivative > 0 and s2.variables[var].derivative < 0:
                return False
            
            if s1.variables[var].derivative < 0 and s2.variables[var].derivative > 0:
                return False
            
        return True

    
def check_variable(s1, s2):
    '''
    to omit the cases (0, 0)->(+, +)
    '''
    for i in s2.variables.keys():
                if s1.variables[i].derivative != s2.variables[i].derivative and (s2.variables[i].magnitude != s1.variables[i].magnitude):
                    return False

    return True

'''
def check_variable(s1, s2):
    for i in s2.variables.keys():
        if s1.variables[i].derivative == 0 and (s2.variables[i].magnitude != s1.variables[i].magnitude):
            return False
    return True
'''

def check_next_state(prev_state, new_state, print_ = False):
    error = False
    

    
    if not(check_variable(prev_state, new_state)):
        #print('hi')
        return False
    
    for var in new_state.variables.values():
        ID = var.id
        DER = var.derivative
        p_plus = [(p.id1, p.id2) for p in list_proportionality if p.type == '+' and p.id2 == ID]
        p_minus = [(p.id1, p.id2) for p in list_proportionality if p.type == '-' and p.id2 == ID]

        l_plus = [(l.id1, l.id2) for l in list_influence if l.type == '+' and l.id2 == ID]
        l_minus = [(l.id1, l.id2) for l in list_influence if l.type == '-' and l.id2 == ID]


        if len(p_plus) == 0 and len(p_minus) == 0 and len(l_minus) == 0 and len(l_plus) == 0:
            if prev_state.variables[ID].derivative > 0:
                if new_state.variables[ID].magnitude < prev_state.variables[ID].magnitude:
                    error = True
                    
            if prev_state.variables[ID].derivative < 0 :
                if new_state.variables[ID].magnitude > prev_state.variables[ID].magnitude:
                    error = True
        
        possible_states = set()

        for (id1, id2) in p_plus:
            
            d = new_state.variables[id1].derivative
            if d == 1:
                if prev_state.variables[id2].derivative == 0 and new_state.variables[id2].derivative != 1:
                    
                    #print('pp')
                    error = True
                elif prev_state.variables[id2].derivative == -1 and new_state.variables[id2].derivative != 0:
                    #print('pp')
                    error = True

        for (id1, id2) in p_minus:
            #print('pm')
            d = new_state.variables[id1].derivative
            if d == 1:
                if prev_state.variables[id2].derivative == 0 and new_state.variables[id2].derivative != -1:
                    error = True
                elif prev_state.variables[id2].derivative == 1 and new_state.variables[id2].derivative != 0:
                    error = True

        for (id1, id2) in l_plus:
            #print('lp')
            m = new_state.variables[id1].magnitude
            if m == 1:
               
                if prev_state.variables[id2].derivative == 0 and new_state.variables[id1].derivative != 1:
                    #print('lp')
                    error = True
                elif prev_state.variables[id2].derivative == -1 and new_state.variables[id1].derivative != 0:
                    #print('lp')
                    error = True

        for (id1, id2) in l_minus:
            
            m = new_state.variables[id1].magnitude
            if m == 1:
                if prev_state.variables[id2].derivative == 0 and new_state.variables[id2].derivative != -1:
                    #print('lm')
                    error = True
                elif prev_state.variables[id2].derivative == 1 and new_state.variables[id2].derivative != 0:
                    #print('lm')
                    error = True
    if error:
        if print_:
            print('State transition impossible!')
        return False
    else:
        if print_:
            print('State transition OK')
        return True

In [235]:
Inflow = variable(1, list_proportionality, list_influence, [0, 1], [-1, 0, 1], [0, -1], 'Inflow')
Outflow = variable(3, list_proportionality, list_influence, [0, 1, 2], [-1, 0, 1], [], 'Outflow')
Volume = variable(2, list_proportionality, list_influence, [0, 1, 2], [-1, 0, 1], [], 'Volume')
#Height = variable(4, list_proportionality, list_influence, [0, 1, 2], [-1, 0, 1], [0, -1], 'Height')
#Pressure = variable(5, list_proportionality, list_influence, [0, 1, 2], [-1, 0, 1], [0, -1], 'Pressure')

System = system({Inflow.id:Inflow, Volume.id:Volume, Outflow.id:Outflow})
System.add_vc(2, 3, 'magnitude', 2)
System.add_vc(2, 3, 'magnitude', 0)

#System.add_vc(2, 4, 'magnitude', 2)
#System.add_vc(2, 4, 'magnitude', 0)

#System.add_vc(2, 5, 'magnitude', 2)
#System.add_vc(2, 5, 'magnitude', 0)

derivatives = []
magnitudes = []

possible_states = []
states = list(System.variables.values())

for s in states:
    
    
    derivatives.append(s.possible_derivative)
    magnitudes.append(s.possible_magnitude)
    p_s = []
    for d in s.possible_derivative:
        for m in s.possible_magnitude:
            r = copy.deepcopy(s)
            r.assign_value(m, d)
            p_s.append(r)
            
        
    possible_states.append(p_s)
            
all_possible_states_of_system = []
for element in itertools.product(*possible_states):
    #print(len(element))
    all_possible_states_of_system.append(system({var.id:var for var in element}))
    #all_possible_states_of_system.append(system((list(element))))
        

In [236]:
for i in range(len(all_possible_states_of_system)):
    all_possible_states_of_system[i].add_vc(2, 3, 'magnitude', 0)
    all_possible_states_of_system[i].add_vc(2, 3, 'magnitude', 2)

    #all_possible_states_of_system[i].add_vc(2, 4, 'magnitude', 2)
    #all_possible_states_of_system[i].add_vc(2, 4, 'magnitude', 0)

    #all_possible_states_of_system[i].add_vc(2, 5, 'magnitude', 2)
    #all_possible_states_of_system[i].add_vc(2, 5, 'magnitude', 0)

In [237]:
j = 0
possible_indices = []
for i in range(len(all_possible_states_of_system)):
    #print(all_possible_states_of_system[i].check_system())
    #print(all_possible_states_of_system[i].check_system())
    if all_possible_states_of_system[i].check_system() == True:
        j+=1
        #print('hi')
        #print(i)
        possible_indices.append(i)
        #all_possible_states_of_system[i].print_system()
        all_possible_states_of_system[i].add_id(i)
        #print(j)
j

32

In [239]:
list_of_states = []
for i in possible_indices:

    k = 0
    for j in possible_indices:

        if check_continuity(all_possible_states_of_system[i], all_possible_states_of_system[j]) and check_next_state(all_possible_states_of_system[i], all_possible_states_of_system[j], print_ = False):
            flag1 = 1
            all_possible_states_of_system[i].add_connection_to(j)
            
        if check_continuity(all_possible_states_of_system[j], all_possible_states_of_system[i]) and check_next_state(all_possible_states_of_system[j], all_possible_states_of_system[i], print_ = False):
            flag2 = 1
            all_possible_states_of_system[i].add_connection_from(j)
        
    #print(len(all_possible_states_of_system[i].connection_to))
    #print(len(all_possible_states_of_system[i].connection_from))
    
    #if len(all_possible_states_of_system[i].connection_to) == 0:

        #print(i, '  NO TO')

        
        
    if len(all_possible_states_of_system[i].connection_from) == 0:
        #print(i, '  NO FROM')
        k = 1

        
    if k != 1:
        list_of_states.append(i)

In [240]:
#for  i in list_of_states:
#    all_possible_states_of_system[i].full_print()

In [241]:
len(list_of_states)

23

In [242]:
to_remove = []
for j in list_of_states:
    flag = 0
    for node in all_possible_states_of_system[j].connection_from:
        if node in list_of_states:
            flag = 1
    if flag == 0 and len(all_possible_states_of_system[j].connection_to) == 0:
        #print(i)
        to_remove.append(j)

        
list_of_states_new = []
for j in list_of_states:
    if j not in to_remove:
        list_of_states_new.append(j)

In [243]:
len(list_of_states_new)

22

In [244]:
def get_valid_states(all_states, list_of_states):
    valid_states = copy.deepcopy(np.array(all_states)[list_of_states])
    for i in range(len(valid_states)):
        to = list(valid_states[i].connection_to)
        from_ = list(valid_states[i].connection_from)
        for j in to:
            if j not in list_of_states:
                valid_states[i].connection_to.remove(j)
        for k in from_:
            if k not in list_of_states:
                valid_states[i].connection_from.remove(k)

    trans = {}
    for i in range(len(list_of_states)):
        trans[list_of_states[i]] = i

    for i in range(len(valid_states)):
        to = list(valid_states[i].connection_to)
        from_ = list(valid_states[i].connection_from)
        new_to = []
        new_from = []
        for j in to:
            try:
                new_to.append(trans[j])
            except KeyError:
                new_to.append(j)
        for k in from_:
            try:
                new_from.append(trans[k])
            except KeyError:
                new_from.append(k)
        valid_states[i].connection_to = set(new_to)
        valid_states[i].connection_from = set(new_from)
    return valid_states

valid_states = get_valid_states(all_possible_states_of_system, list_of_states_new)

In [245]:
import pygraphviz as pgv

graph = pgv.AGraph(directed=True, fixedsize=True)

def graph_state(state):
    
    def tm(val): #translate magnitude
        if val == 1:
            return '+'
        elif val == 2:
            return 'max'
        else:
            return val
    def td(val): #translate derivative
        if val == 1:
            return '+'
        elif val == -1:
            return '-'
        else:
            return val
        
    return 'I(%s, %s)\nV(%s, %s)\nO(%s, %s)' % (tm(state.variables[1].magnitude), \
                                               td(state.variables[1].derivative), \
                                               tm(state.variables[2].magnitude), \
                                               td(state.variables[2].derivative), \
                                               tm(state.variables[3].magnitude), \
                                               td(state.variables[3].derivative)) 

for i in range(len(valid_states)):
    graph.add_node(graph_state(valid_states[i]), color='red', style = 'filled, bold')
num_edges = 0
for i in range(len(valid_states)):
    for j in valid_states[i].connection_from:
        num_edges+=1
        graph.add_edge(graph_state(valid_states[j]), graph_state(valid_states[i]), timestep = True)

print('#edges ', num_edges)        
graph.layout(prog='dot')

graph.draw('graph1_to.png')

#edges  94


In [246]:
def make_intra_trace(s):
        #print('STATE ID : ', s.id)
        s.print_system()
        for s1 in s.variables.values():
            if s1.name == 'Inflow':
                In = s1
            if s1.name == 'Outflow':
                Out = s1
            if s1.name == 'Volume':
                V = s1
            
        
        if In.magnitude == 1:
            i1 = 'THE WATER IS FLOWING FROM THE TAP, '
        if In.magnitude == 0:
            i1 = 'THERE IS NO WATER IN A TAP, '
            
        if In.derivative == 0:  
            i2 = 'WE DONT CHANGE THE RATE OF INFLOW'
        if In.derivative == -1:  
            i2 = 'WE DECREASE THE RATE OF INFLOW'
        if In.derivative == 1:  
            i2 = 'WE INCREASE THE RATE OF INFLOW'
        
        if V.magnitude == 0:
            v1 = 'THERE IS NO WATER IN A CONTAINER, '
        if V.magnitude == 1:
            v1 = 'THERE IS WATER IN A CONTAINER, '
        if V.magnitude == 2:
            v1 = 'THE RATE OF THE WATER IN A CONTAINER IS AT ITS MAX, '
            
        if V.derivative == 0:  
            v2 = 'THE AMOUNT OF WATER DOES NOT CHANGE'
        if V.derivative == -1:  
            v2 = 'THE AMOUNT OF WATER IS DECREASING'
        if V.derivative == 1:  
            v2 = 'THE AMOUNT OF WATER IS INCREASING'
            
            
        if Out.magnitude == 0:
            o1 = 'THE WATER DOES NOT FLOW OUT FROM A CONTAINER, '
        if Out.magnitude == 1:
            o1 = 'THE WATER FLOWS OUT FROM A CONTAINER, '
        if Out.magnitude == 2:
            o1 = 'THE RATE OF THE WATER FLOWING OUT FROM THE CONTAINER IS AT ITS MAX , '
            
        if Out.derivative == 0:  
            o2 = 'THE AMOUNT OF FLOWING OUT WATER DOES NOT CHANGE'
        if Out.derivative == -1:  
            o2 = 'THE AMOUNT OF FLOWING OUT WATER IS DECREASING'
        if Out.derivative == 1:  
            o2 = 'THE AMOUNT OF FLOWING OUT WATER IS INCREASING'
            
        print(i1, i2)
        print(v1, v2)
        print(o1, o2, '\n')
            

In [211]:
def make_inter_trace(s1, s2):
        #print('STATE ID : ', s.id)
        #print('The current state is: ', s1.id, ' the possible transition to the state ', s2.id)
        print('The current state is: ')
        s1.print_system()
        print('The possible transition to the state: ')
        s2.print_system()
        #s2.print_system()
        for v in s1.variables.values():
            if v.name == 'Inflow':
                In1 = v
            if v.name == 'Outflow':
                Out1 = v
            if v.name == 'Volume':
                V1 = v
                
        for v in s2.variables.values():
            if v.name == 'Inflow':
                In2 = v
            if v.name == 'Outflow':
                Out2 = v
            if v.name == 'Volume':
                V2 = v
                
        #print('From the state ', s1.id,' such transitions are possible: ', s1.connection_to)
        
        if In1.derivative == In2.derivative:
            d1 = 'The rate of water in a tap does not change, '
        if In1.derivative > In2.derivative:
            d1 = 'The rate of water flowing in is getting smaller, '
        if In1.derivative < In2.derivative:
            d1 = 'The rate of water flowing in is getting bigger, '
            
        if In1.magnitude > In2.magnitude:
            d2 = 'the water starts to flow in.'
        if In1.magnitude < In2.magnitude:
            d2 = 'the water stops flowing in.'
        if In1.magnitude == In2.magnitude:
            d2 = 'the amount of water flowing in stays the same.'
        
        if Out1.derivative == Out2.derivative:
            d3 = 'The rate of water flowing out does not change, '
        if Out1.derivative > Out2.derivative:
            d3 = 'The rate of water flowing out decreases, '
        if Out1.derivative < Out2.derivative:
            d3 = 'The rate of water flowing out increases, '   
            
        if Out1.magnitude < Out2.magnitude:
            d4 = 'the amount of water flowing out increases.'
        if Out1.magnitude > Out2.magnitude:
            d4 = 'the amount of water flowing out decreases.'
        if Out1.magnitude == Out2.magnitude:
            d4 = 'the amount of water flowing out stays the same.'
        
        if V1.derivative < V2.derivative:
            d5 = 'and the container fills up faster. '
        if V1.derivative > V2.derivative:
            d5 = 'and the container fills up slower. '
        if V1.derivative == V2.derivative:
            d5 = '.'
        #    d5 = 'The container fills up with the same speed'
            
        if V1.magnitude < V2.magnitude:
            d6 = 'The amount of water in a container increased '
        if V1.magnitude > V2.magnitude:
            d6 = 'The amount of water in a container dropped '
        if V1.magnitude == V2.magnitude:
            d6 = 'The amount of water in a container stays the same '
            
        #if V1.derivative == V2.derivative:
        #    d5 = 'The amount of water in a container stays the same. '
        print(d1, d2)
        print(d3, d4)
        print(d6, d5, '\n')

In [249]:
for i in list_of_states_new: 
    print('ID ', all_possible_states_of_system[i].id)
    make_intra_trace(all_possible_states_of_system[i])

ID  111
SYSTEM: 
Inflow  M:  1  D:  -1
Volume  M:  0  D:  0
Outflow  M:  0  D:  0
THE WATER IS FLOWING FROM THE TAP,  WE DECREASE THE RATE OF INFLOW
THERE IS NO WATER IN A CONTAINER,  THE AMOUNT OF WATER DOES NOT CHANGE
THE WATER DOES NOT FLOW OUT FROM A CONTAINER,  THE AMOUNT OF FLOWING OUT WATER DOES NOT CHANGE 

ID  138
SYSTEM: 
Inflow  M:  1  D:  -1
Volume  M:  0  D:  1
Outflow  M:  0  D:  0
THE WATER IS FLOWING FROM THE TAP,  WE DECREASE THE RATE OF INFLOW
THERE IS NO WATER IN A CONTAINER,  THE AMOUNT OF WATER IS INCREASING
THE WATER DOES NOT FLOW OUT FROM A CONTAINER,  THE AMOUNT OF FLOWING OUT WATER DOES NOT CHANGE 

ID  141
SYSTEM: 
Inflow  M:  1  D:  -1
Volume  M:  0  D:  1
Outflow  M:  0  D:  1
THE WATER IS FLOWING FROM THE TAP,  WE DECREASE THE RATE OF INFLOW
THERE IS NO WATER IN A CONTAINER,  THE AMOUNT OF WATER IS INCREASING
THE WATER DOES NOT FLOW OUT FROM A CONTAINER,  THE AMOUNT OF FLOWING OUT WATER IS INCREASING 

ID  175
SYSTEM: 
Inflow  M:  0  D:  0
Volume  M:  1  D:

In [248]:
for i in list_of_states_new:
    s1 = all_possible_states_of_system[i]

    for j in s1.connection_to:
        s2 = all_possible_states_of_system[j]
        make_inter_trace(s1, s2)

The current state is: 
SYSTEM: 
Inflow  M:  1  D:  -1
Volume  M:  0  D:  1
Outflow  M:  0  D:  0
The possible transition to the state: 
SYSTEM: 
Inflow  M:  1  D:  0
Volume  M:  0  D:  0
Outflow  M:  0  D:  0
The rate of water flowing in is getting bigger,  the amount of water flowing in stays the same.
The rate of water flowing out does not change,  the amount of water flowing out stays the same.
The amount of water in a container stays the same  and the container fills up slower.  

The current state is: 
SYSTEM: 
Inflow  M:  1  D:  -1
Volume  M:  0  D:  1
Outflow  M:  0  D:  0
The possible transition to the state: 
SYSTEM: 
Inflow  M:  1  D:  0
Volume  M:  0  D:  1
Outflow  M:  0  D:  1
The rate of water flowing in is getting bigger,  the amount of water flowing in stays the same.
The rate of water flowing out increases,  the amount of water flowing out stays the same.
The amount of water in a container stays the same  . 

The current state is: 
SYSTEM: 
Inflow  M:  1  D:  -1
Volume

Inflow  M:  1  D:  0
Volume  M:  1  D:  0
Outflow  M:  1  D:  0
The possible transition to the state: 
SYSTEM: 
Inflow  M:  1  D:  1
Volume  M:  0  D:  0
Outflow  M:  0  D:  0
The rate of water flowing in is getting bigger,  the amount of water flowing in stays the same.
The rate of water flowing out does not change,  the amount of water flowing out decreases.
The amount of water in a container dropped  . 

The current state is: 
SYSTEM: 
Inflow  M:  1  D:  0
Volume  M:  1  D:  0
Outflow  M:  1  D:  0
The possible transition to the state: 
SYSTEM: 
Inflow  M:  0  D:  0
Volume  M:  2  D:  0
Outflow  M:  2  D:  0
The rate of water in a tap does not change,  the water starts to flow in.
The rate of water flowing out does not change,  the amount of water flowing out increases.
The amount of water in a container increased  . 

The current state is: 
SYSTEM: 
Inflow  M:  1  D:  0
Volume  M:  0  D:  1
Outflow  M:  0  D:  0
The possible transition to the state: 
SYSTEM: 
Inflow  M:  0  D:  0
V

Outflow  M:  2  D:  0
The rate of water flowing in is getting smaller,  the amount of water flowing in stays the same.
The rate of water flowing out does not change,  the amount of water flowing out stays the same.
The amount of water in a container stays the same  . 

The current state is: 
SYSTEM: 
Inflow  M:  0  D:  1
Volume  M:  2  D:  0
Outflow  M:  2  D:  0
The possible transition to the state: 
SYSTEM: 
Inflow  M:  0  D:  1
Volume  M:  2  D:  0
Outflow  M:  2  D:  0
The rate of water in a tap does not change,  the amount of water flowing in stays the same.
The rate of water flowing out does not change,  the amount of water flowing out stays the same.
The amount of water in a container stays the same  . 

The current state is: 
SYSTEM: 
Inflow  M:  0  D:  1
Volume  M:  2  D:  0
Outflow  M:  2  D:  0
The possible transition to the state: 
SYSTEM: 
Inflow  M:  1  D:  1
Volume  M:  2  D:  0
Outflow  M:  2  D:  0
The rate of water in a tap does not change,  the water stops flowing in

In [212]:
#i = list_of_states_new[21]
i = 435
all_possible_states_of_system[i].full_print()

s1 = all_possible_states_of_system[i]

for j in s1.connection_to:
        s2 = all_possible_states_of_system[j]
        #s2.full_print()
        make_inter_trace(s1, s2)

SYSTEM: 
Inflow  M:  1  D:  1
Volume  M:  0  D:  0
Outflow  M:  0  D:  0
From  435  such transitions are possible:  {465, 435}
To  435  are possible transitions from :  {354, 300, 364, 462, 303, 273, 465, 435, 283, 445}
The current state is: 
SYSTEM: 
Inflow  M:  1  D:  1
Volume  M:  0  D:  0
Outflow  M:  0  D:  0
The possible transition to the state: 
SYSTEM: 
Inflow  M:  1  D:  1
Volume  M:  0  D:  1
Outflow  M:  0  D:  1
The rate of water in a tap does not change,  the amount of water flowing in stays the same.
The rate of water flowing out increases,  the amount of water flowing out stays the same.
The amount of water in a container stays the same  and the container fills up faster.  

The current state is: 
SYSTEM: 
Inflow  M:  1  D:  1
Volume  M:  0  D:  0
Outflow  M:  0  D:  0
The possible transition to the state: 
SYSTEM: 
Inflow  M:  1  D:  1
Volume  M:  0  D:  0
Outflow  M:  0  D:  0
The rate of water in a tap does not change,  the amount of water flowing in stays the same.
T

In [204]:
i = list_of_states_new[20]
all_possible_states_of_system[i].full_print()

SYSTEM: 
Inflow  M:  1  D:  1
Volume  M:  0  D:  1
Outflow  M:  0  D:  0
From  462  such transitions are possible:  {273, 435, 465, 303}
To  462  are possible transitions from :  {465, 303}
